<a href="https://colab.research.google.com/github/baut-jc/DDDS-My-Projects/blob/main/Project-05/Project5_02_NLP_Description_for_Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture output
#install Wikipedia API
!pip3 install wikipedia-api

In [34]:
import pandas as pd
import numpy as np
import re

from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import wikipediaapi
import ipywidgets as widgets # ipywidgets are used for creating interactive user interface elements in Jupyter Notebooks.
import nltk

nltk.download('punkt_tab')
pd.options.display.max_columns = 100


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


##Data Collection and Sources

In [ ]:
df=pd.read_csv('https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv')

In [ ]:
df.shape, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42786 entries, 0 to 42785
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URI     42786 non-null  object
 1   name    42786 non-null  object
 2   text    42786 non-null  object
dtypes: object(3)
memory usage: 1002.9+ KB


((42786, 3), None)

In [ ]:
df.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [ ]:
df['name'].unique()[:10] # Displaying first 10 for brevity

array(['Digby Morrell', 'Alfred J. Lewy', 'Harpdog Brown',
       'Franz Rottensteiner', 'G-Enka', 'Sam Henderson', 'Aaron LaCrate',
       'Trevor Ferguson', 'Grant Nelson', 'Cathy Caruth'], dtype=object)

## Data Cleaning

In [ ]:
# Making a copy of 'original_column' (name) for potential later use.
df['copied_name'] = df['name'].copy()

In [ ]:
# Create a copy of the DataFrame to store cleaned text, preserving the original.
df_cleaned = df.copy()

In [ ]:
# Select the target person
# target_names = df_cleaned['name'].sample() #random famous person

target_name = "Dave Bautista" # --> just another 'Bautista ;)'
target_index = df_cleaned[df['name'] == target_name].index[0]

In [ ]:
# Define the `clean_text` function to preprocess text strings. --> 2b-Text.Representation
def clean_text(text):
    text = str(text).lower()                            # Convert text to string and lowercase for normalization (2b-Text.Representation.end (1).ipynb)
    text = re.sub(r'\[.*?\]', '', text)                 # Remove text within square brackets (e.g., citations)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)   # Remove URLs
    text = re.sub(r'<.*?>+', '', text)                  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'), '', text)  # Remove punctuation (general text cleaning, aligns with concepts in 2b-Text.Representation.end (1).ipynb on tokenization)
    text = re.sub(r'\n', '', text)                      # Remove newline characters
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [ ]:
# Apply the cleaning function to the 'text' column to create 'cleaned_text'.
df_cleaned['Overview'] = df_cleaned['text'].apply(clean_text)

# Natural Language Processing



This project will give you practical experience using Natural Language Processing techniques. This project is in three parts:
- in part 1) you will use a dataset in a CSV file
- in part 2) you will use the Wikipedia API to directly access content
on Wikipedia.
- in part 3) you will make your notebook interactive


## Part 1: Processing DS for NLP

- The CSV file is available at https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv
- The file contains a list of famous people and a brief overview.
- The goal of part 1) is to ...
  1. Pick one person from the list ( the target person ) and output 10 other people who's overview are "closest" to the target person in a Natural Language Processing sense
  1. Also output the sentiment of the overview of the target person



### 1.1 Natural Language Processing

In [12]:
# ----> Bag of Words
# Perform the count transformation
vectorizer = CountVectorizer(stop_words='english')
bow_vec = vectorizer.fit_transform(df_cleaned['Overview'])
bow_vec # <Compressed Sparse Row sparse matrix of dtype 'int64' with 5372283 stored elements and shape (42786, 397235)>
# bow_vec.toarray() ---> too dense for the RAM with the .shape of things!!

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 5372283 stored elements and shape (42786, 397235)>

In [13]:
# Perform the TF-IDF transformation - Option 1 (TfidfVectorizer)
tf_idf_vec = TfidfVectorizer(stop_words = 'english')
tf_idf_fame = tf_idf_vec.fit_transform(df_cleaned['Overview'])
tf_idf_fame

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5372283 stored elements and shape (42786, 397235)>

In [14]:
# Get a data frame with the TF-IDF values sorted for document 0
df = pd.DataFrame(tf_idf_fame[0].T.todense(), index=tf_idf_vec.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df

,TF-IDF
morrell,0.568121
football,0.394940
kangaroos,0.261344
club,0.177284
carlton,0.140973
...,...
galion,0.000000
galinsky,0.000000
galinhas,0.000000
galing,0.000000


In [15]:
# # Initialize TF-IDF Vectorizer as demonstrated in 2b-Text.Representation.end (1).ipynb and 4a-NLP.KNN.clustering.end (1).ipynb
# tfidf_vectorizer = TfidfVectorizer(
#     min_df=5,           # Filter out terms that appear in less than 5 documents (2b-Text.Representation.end (1).ipynb)
#     max_df=0.8,         # Filter out terms that appear in more than 80% of documents (2b-Text.Representation.end (1).ipynb)
#     stop_words='english', # Remove common English stop words (2b-Text.Representation.end (1).ipynb)
#     ngram_range=(1,2)   # Include both unigrams and bigrams for richer context (2b-Text.Representation.end (1).ipynb, 4a-NLP.KNN.clustering.end (1).ipynb)
# )
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_cleaned['cleaned_text'])

# tfidf_matrix.shape

In [16]:
# Pick one person from the list ( the target person )
target_text = df_cleaned.iloc[target_index]['Overview']

In [17]:
#  K-Nearest --> 10 other people who's overview are "closest" to the target person
nn = NearestNeighbors().fit(tf_idf_fame)

In [18]:
fame_person = tf_idf_fame[target_index]
fame_person.shape

(1, 397235)

In [19]:
distances, indices = nn.kneighbors(
  X = fame_person,
  n_neighbors = 11,
)

distances, indices

(array([[0.        , 1.09164226, 1.09427749, 1.09709832, 1.10704627,
         1.10878842, 1.11770999, 1.11959947, 1.12000911, 1.14675295,
         1.14697472]]),
 array([[ 5327,  6879, 29541, 32332, 15787, 42003, 25656, 33284, 38664,
         36932, 13862]]))

In [36]:
for i, idx in enumerate(indices[0]):
        if idx != target_index:  # Exclude the target person itself, as their distance to themselves is 0.
            similarity = 1 - distances[0][i]  # Convert cosine distance to similarity score (3c-K-Nearest.Neighbors.end (1).ipynb)
            print(f"  {df_cleaned.iloc[idx]['name']} (Similarity Score: {similarity:.4f})")

  CM Punk (Similarity Score: -0.0916)
  John Cena (Similarity Score: -0.0943)
  Edge (wrestler) (Similarity Score: -0.0971)
  Rey Mysterio (Similarity Score: -0.1070)
  The Miz (Similarity Score: -0.1088)
  Randy Orton (Similarity Score: -0.1177)
  Big Show (Similarity Score: -0.1196)
  Daniel Bryan (Similarity Score: -0.1200)
  Beth Phoenix (Similarity Score: -0.1468)
  Kane (wrestler) (Similarity Score: -0.1470)


In [39]:
print(df_cleaned.iloc[idx]['name'])

Kane (wrestler)


### 1.3.3 Sentiment

In [21]:
df_cleaned.iloc[target_index]['name']

'Dave Bautista'

In [22]:
sentiment = TextBlob(target_text).sentiment
f'Sentiment for {target_name} is {sentiment}!'

'Sentiment for Dave Bautista is Sentiment(polarity=0.13480392156862744, subjectivity=0.20735294117647057)!'

## Part 2: Using Wikipedia API



- For the same target person that you chose in Part 1), use the Wikipedia API to access the whole content of the target person's Wikipedia page.
- The goal of Part 2) is to ...
  1. Print out the text of the Wikipedia article for the target person
  1. Determine the sentiment of the text of the Wikipedia page for the target person
  1. Collect the text of the Wikipedia pages from the 10 nearest neighbors from Part 1)
  1. Determine the nearness ranking of these 10 people to your target person based on their entire Wikipedia page
  1. Compare, i.e. plot,  the nearest ranking from Step 1) with the Wikipedia page nearness ranking.  A difference of the rank is one means of comparison.



### 2.1 Target's Article

In [23]:
# Pull out the Dave Bautista page from wikipedia - https://en.wikipedia.org/wiki/
topic = 'Dave Bautista'
wikip = wikipediaapi.Wikipedia(user_agent = 'foobar')
page_ex = wikip.page(topic)
wiki_text = page_ex.text
wiki_text

'David Michael Bautista Jr. (born January 18, 1969) is an American actor and retired professional wrestler. Regarded as one of the most prolific professional wrestlers of his generation, he rose to fame for his multiple stints in WWE between 2002 and 2019.\nBautista began his wrestling career in 1999 and signed with WWE (then WWF) in 2000. From 2002 to 2010, he gained fame under the ring name Batista, initially as a member of Evolution. He would go on to win the WWE Championship twice, the World Heavyweight Championship four times (with his first reign remaining the longest in history at 282 days), the World Tag Team Championship three times (twice with Ric Flair and once with John Cena), and the WWE Tag Team Championship once (with Rey Mysterio). He also won the 2005 and 2014 Royal Rumble matches and subsequently headlined WrestleMania 21 and WrestleMania XXX, with the former being one of the top five highest-grossing PPV events in wrestling history. He retired from professional wrest

In [24]:
# Replace newline chars with spaces before doing any processing. Strip the ' and "s" from possessives
wiki_text_clean = (
    wiki_text
    .lower()
    .replace("\n"," ")
    .replace("'s",'')
    .replace("'",'')
)
wiki_text_clean

'david michael bautista jr. (born january 18, 1969) is an american actor and retired professional wrestler. regarded as one of the most prolific professional wrestlers of his generation, he rose to fame for his multiple stints in wwe between 2002 and 2019. bautista began his wrestling career in 1999 and signed with wwe (then wwf) in 2000. from 2002 to 2010, he gained fame under the ring name batista, initially as a member of evolution. he would go on to win the wwe championship twice, the world heavyweight championship four times (with his first reign remaining the longest in history at 282 days), the world tag team championship three times (twice with ric flair and once with john cena), and the wwe tag team championship once (with rey mysterio). he also won the 2005 and 2014 royal rumble matches and subsequently headlined wrestlemania 21 and wrestlemania xxx, with the former being one of the top five highest-grossing ppv events in wrestling history. he retired from professional wrestl

In [25]:
# Break up single string into separate sentences
wiki_blob = TextBlob(wiki_text_clean)
len(wiki_blob.sentences)

283

In [26]:
# Only look at first 20 sentences
target_blob = wiki_blob.sentences[0:20]
target_blob

[Sentence("david michael bautista jr. (born january 18, 1969) is an american actor and retired professional wrestler."),
 Sentence("regarded as one of the most prolific professional wrestlers of his generation, he rose to fame for his multiple stints in wwe between 2002 and 2019. bautista began his wrestling career in 1999 and signed with wwe (then wwf) in 2000. from 2002 to 2010, he gained fame under the ring name batista, initially as a member of evolution."),
 Sentence("he would go on to win the wwe championship twice, the world heavyweight championship four times (with his first reign remaining the longest in history at 282 days), the world tag team championship three times (twice with ric flair and once with john cena), and the wwe tag team championship once (with rey mysterio)."),
 Sentence("he also won the 2005 and 2014 royal rumble matches and subsequently headlined wrestlemania 21 and wrestlemania xxx, with the former being one of the top five highest-grossing ppv events in wr

### 2.2 Target's Sentiment

In [27]:
# Sentiment Analysis using the same function from Part 1 (get_sentiment from 1b-Intro.to.NLP.using.TextBlob.end (1).ipynb)
sentiment = TextBlob(wiki_text_clean).sentiment
f'{sentiment} for {target_name}!'

'Sentiment(polarity=0.059599167773986696, subjectivity=0.36244615556209775) for Dave Bautista!'

### 2.3 K-Nearest Neighbors

In [41]:
neighbor_wiki_texts = {target_name_part1: topic} # Include target in corpus


NameError: name 'target_name_part1' is not defined

In [35]:
neighbor_titles = [
   "David Nelson (electronic sports player)",
  "John Nelson (conductor)",
  "Sammy Nelson",
  "Michael R. Nelson",
  "Tyrone Nelson",
  "Ben Nelson (Australian footballer)",
  "Jeff Nelson (pitcher)",
  "Nelson Quan",
  "Chris Grayston",
  "Roger Nelson (baseball)"
]
# Text cleaning function
def clean_wikipedia_text(text):
    text = re.sub(r'\[\d+\]', '', text)
    text = re.split(r'==\s*References\s*==', text)[0]
    text = re.sub(r'==+.*?==+', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Fetch and clean pages
titles = [topic] + neighbor_titles
texts = []
valid_titles = []

for title in titles:
    page = wikip.page(title)
    if page.exists():
        cleaned = clean_wikipedia_text(page.text)
        texts.append(cleaned)
        valid_titles.append(title)
    else:
        print(f"Page not found: {title}")

# Compute TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)

# Compute cosine similarities
cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]  # similarities to target

# Rank by similarity
ranked = sorted(zip(valid_titles[1:], cos_sim), key=lambda x: x[1], reverse=True)

# Print first 500 characters of each cleaned article
print("\n--- First 500 Characters of Each Cleaned Article ---\n")
for title, text in zip(valid_titles, texts):
    print(f"\n--- {title} ---\n")
    print(text[:500])  # Print first 500 characters


# Print ranked results
print("\n--- Similarity Ranking to Target: Grant Nelson (DJ) ---\n")
for i, (title, score) in enumerate(ranked, 1):
    print(f"{i}. {title}: {score:.4f}")


--- First 500 Characters of Each Cleaned Article ---


--- Dave Bautista ---

David Michael Bautista Jr. (born January 18, 1969) is an American actor and retired professional wrestler. Regarded as one of the most prolific professional wrestlers of his generation, he rose to fame for his multiple stints in WWE between 2002 and 2019. Bautista began his wrestling career in 1999 and signed with WWE (then WWF) in 2000. From 2002 to 2010, he gained fame under the ring name Batista, initially as a member of Evolution. He would go on to win the WWE Championship twice, the World H

--- David Nelson (electronic sports player) ---

David Nelson (born January 18, 1974, in New Hampshire) is an American arcade video game player who holds world record high scores listed in the 2008 Guinness World Records-Gamer's Edition. David Nelson has broken many world records while competing in classic arcade championships at the Funspot Family Fun Center in Weirs Beach, New Hampshire. As of 2008, David Nelson h

In [29]:
target_tfidf_vector = tfidf_matrix[target_index]
target_index, target_tfidf_vector

NameError: name 'tfidf_matrix' is not defined

In [28]:
similar_people_data = []
count = 0 # Iterate through the neighbors, excluding the target person themselves

for i, idx in enumerate(indices[0]):
            if idx != target_index and count < 10:
                similarity = 1 - distances[0][i] # Cosine similarity = 1 - Cosine distance
                similar_person_name = df.iloc[idx]['name']
                print(f"  - {similar_person_name} (Similarity Score: {similarity:.4f})")
                similar_people_data.append({'name': similar_person_name, 'original_rank': count + 1})
                count += 1
            if count >= 10: # Ensure we only get top 10
                break

nearest_df = pd.DataFrame(similar_people_data) # Return the DataFrame of neighbors with their ranks

KeyError: 'name'

In [ ]:
target_topic = wiki_blob.iloc[topic]['Overview']

In [ ]:
nn = NearestNeighbors().fit(tf_idf_fame)

In [ ]:
fame_person = tf_idf_fame[target_index]
fame_person.shape

In [ ]:
distances, indices = nn.kneighbors(
  X = fame_person,
  n_neighbors = 11,
)

distances, indices

In [ ]:
for i, idx in enumerate(indices[0]):
        if idx != target_index:  # Exclude the target person itself, as their distance to themselves is 0.
            similarity = 1 - distances[0][i]  # Convert cosine distance to similarity score (3c-K-Nearest.Neighbors.end (1).ipynb)
            print(f"  {df_cleaned.iloc[idx]['name']} (Similarity Score: {similarity:.4f})")

In [ ]:
for i, idx in enumerate(indices[0]):
        if idx != target_index:  # Exclude the target person itself, as their distance to themselves is 0.
            similarity = 1 - distances[0][i]  # Convert cosine distance to similarity score (3c-K-Nearest.Neighbors.end (1).ipynb)
            print(f"  {wiki_blob.iloc[idx]['name']} (Similarity Score: {similarity:.4f})")

### Part 3)


Make an interactive notebook where a user can choose or enter a name and the notebook displays the 10 closest individuals.

In addition to presenting the project slides, at the end of the presentation each student will demonstrate their code using a famous person suggested by the other students that exists in the DBpedia set.


In [ ]:
!curl -s https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv | wc -l